In [2]:
import numpy as np
import pandas as pd

file_path1 = 'C:/temp/X_train.csv';file_path2 = 'C:/temp/y_train.csv'
file_path3 = 'C:/temp/X_test.csv'; file_path4 = 'C:/temp/y_test.csv'
X1 = pd.read_csv(file_path1);y1 = pd.read_csv(file_path2)
X2 = pd.read_csv(file_path3);y2 = pd.read_csv(file_path4)
X_train = np.array(X1);X_test = np.array(X2)
y_train = np.array(y1).flatten();y_test = np.array(y2).flatten()

In [15]:
X_train[1:20]

array([[ 0.79692948, -0.25628095],
       [-0.11010756, -0.37411937],
       [ 0.3224064 ,  0.9704698 ],
       [-1.27629731, -0.3407025 ],
       [ 0.65889623, -0.2712306 ],
       [-0.23960413, -0.31695894],
       [ 0.18617211,  0.72599904],
       [-0.75126704, -0.29761233],
       [-0.75889266, -0.37851633],
       [-1.37136406, -0.35301398],
       [-1.20959179, -0.36444607],
       [-1.3035699 , -0.37851633],
       [-0.4287365 , -0.28266268],
       [ 0.02270382, -0.3512552 ],
       [-0.95989197,  1.92021232],
       [-0.93703827, -0.30640625],
       [-1.23464078, -0.3204765 ],
       [-0.54648709,  1.33453776],
       [-1.50989402, -0.38643085]])

In [5]:
# numpy only data peek

def data_info(data):
    missing_values = np.isnan(data).sum()
    size_in_mb = data.nbytes / (1024 ** 2)
    if len(data.shape) > 1:
        mean = data.mean()
        sd = data.std()
    else:
        mean = np.mean(data)
        sd = np.std(data)
    return missing_values, size_in_mb, mean, sd

def data_summary(X_train, y_train, X_test, y_test):
    header = "+------------+-------------------------+------------------+--------------+---------+-----------+------------+"
    print(header)
    print("| Data       | Type                    | Shape            | MissingVals  | Size(MB)| Mean      | SD         |")
    print(header)

    for name, data in [("X_train", X_train), ("y_train", y_train), ("X_test", X_test), ("y_test", y_test)]:
        missing_values, size_in_mb, mean, sd = data_info(data)
        print("| {:<9} | {:<24} | {:<16} | {:<12} | {:<7.2f} | {:<9.2f} | {:<10.2f} |".format(
            name, str(type(data)), str(data.shape), missing_values, size_in_mb, mean, sd))
    print(header)


data_summary(X_train, y_train, X_test, y_test)

+------------+-------------------------+------------------+--------------+---------+-----------+------------+
| Data       | Type                    | Shape            | MissingVals  | Size(MB)| Mean      | SD         |
+------------+-------------------------+------------------+--------------+---------+-----------+------------+
| X_train   | <class 'numpy.ndarray'>  | (10400792, 2)    | 0            | 158.70  | 0.00      | 0.99       |
| y_train   | <class 'numpy.ndarray'>  | (10400792,)      | 0            | 79.35   | 0.66      | 0.47       |
| X_test    | <class 'numpy.ndarray'>  | (2764768, 2)     | 0            | 42.19   | -0.01     | 1.05       |
| y_test    | <class 'numpy.ndarray'>  | (2764768,)       | 0            | 21.09   | 0.64      | 0.48       |
+------------+-------------------------+------------------+--------------+---------+-----------+------------+


In [6]:
# Binary target: logisitic regression which is acutally classificaiton.

class LogisticRegression:

    def __init__(self, lr=0.01, num_iter=1000):
        self.lr = lr
        self.num_iter = num_iter

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        # initialization of weights
        self.theta = np.zeros(X.shape[1])
        self.bias = 0

        for i in range(self.num_iter):
            model = np.dot(X, self.theta) + self.bias
            predictions = self.sigmoid(model)

            # compute gradient
            d_theta = (1 / len(X)) * np.dot(X.T, (predictions - y))
            d_bias = (1 / len(X)) * np.sum(predictions - y)

            # update my weights
            self.theta -= self.lr * d_theta
            self.bias -= self.lr * d_bias

    def predict_prob(self, X):
        return self.sigmoid(np.dot(X, self.theta) + self.bias)

    def predict(self, X, threshold=0.5):
        return (self.predict_prob(X) >= threshold).astype(int)

In [7]:
def confusion_matrix(y_true, y_pred):
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))
    return TP, TN, FP, FN

def accuracy(TP, TN, FP, FN):
    return (TP + TN) / (TP + TN + FP + FN)

def specificity(TN, FP):
    return TN / (TN + FP)

def sensitivity(TP, FN):
    return TP / (TP + FN)

def f1_score(TP, FP, FN):
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    return 2 * (precision * recall) / (precision + recall)

def print_confusion_matrix(TP, TN, FP, FN):
    head_cm = "+-----------------+------------+"
    print(head_cm)
    print("|                 | Predicted  |")
    print("| Actual          | 0    | 1    |")
    print(head_cm)
    print("| 0               | {:<5} | {:<5} |".format(TN, FP))
    print("| 1               | {:<5} | {:<5} |".format(FN, TP))
    print(head_cm)
    print()

def metrics(y_true, y_pred):
    TP, TN, FP, FN = confusion_matrix(y_true, y_pred)
    acc = accuracy(TP, TN, FP, FN)
    spec = specificity(TN, FP)
    sens = sensitivity(TP, FN)
    f1 = f1_score(TP, FP, FN)

    print_confusion_matrix(TP, TN, FP, FN)

    head = "+--------------+---------+"
    print(head)
    print("| Metric       | Value   |")
    print(head)
    print("| Accuracy     | {:.4f} |".format(acc))
    print("| Specificity  | {:.4f} |".format(spec))
    print("| Sensitivity  | {:.4f} |".format(sens))
    print("| F1 Score     | {:.4f} |".format(f1))
    print(head)

In [9]:
model_logreg = LogisticRegression(lr=0.01, num_iter=10)
model_logreg.fit(X_train, y_train)
y_pred_logreg = model_logreg.predict(X_test)
metrics(y_test, y_pred_logreg)

+-----------------+------------+
|                 | Predicted  |
| Actual          | 0    | 1    |
+-----------------+------------+
| 0               | 0     | 982339 |
| 1               | 0     | 1782429 |
+-----------------+------------+

+--------------+---------+
| Metric       | Value   |
+--------------+---------+
| Accuracy     | 0.6447 |
| Specificity  | 0.0000 |
| Sensitivity  | 1.0000 |
| F1 Score     | 0.7840 |
+--------------+---------+


In [10]:
# grad desect method

def calc_cost(X, y, theta):
    m = len(y)
    cost = (1/2*m) * np.sum(np.square(X.dot(theta) - y))
    return cost

def grad_desc(X, y, theta, lr, iters):
    m = len(y)
    cost_hist = np.zeros(iters)

    for i in range(iters):
        theta = theta - (1/m) * lr * (X.T.dot(X.dot(theta) - y))
        cost_hist[i] = calc_cost(X, y, theta)

    return theta, cost_hist

# initial coeffx w/intercept)
theta = np.zeros(X_train.shape[1])

#  gradient-descent settings
iters = 500
lr = 0.0001

# Run gradient-descent
theta, cost_hist = grad_desc(X_train, y_train, theta, lr, iters)

pred = X_test.dot(theta)
mae = np.mean(np.abs(pred - y_test))
mse = np.mean((pred - y_test)**2)
rmse = np.sqrt(np.mean((pred - y_test)**2))
ss_res = np.sum((pred - y_test)**2)
ss_tot = np.sum((y_test - np.mean(y_test))**2)
r2 = 1 - (ss_res / ss_tot)
print("mean squared error: ",mse*100)
print("mean absolute error: ",mae*100)
print("mean root mean error: ",rmse*100)
print("r2 value: ",r2)

mean squared error:  64.35492876641504
mean absolute error:  64.47533782344999
mean root mean error:  80.22152377411878
r2 value:  -1.809477617267969


In [11]:
metrics(y_test, pred)

+-----------------+------------+
|                 | Predicted  |
| Actual          | 0    | 1    |
+-----------------+------------+
| 0               | 0     | 0     |
| 1               | 0     | 0     |
+-----------------+------------+

+--------------+---------+
| Metric       | Value   |
+--------------+---------+
| Accuracy     | nan |
| Specificity  | nan |
| Sensitivity  | nan |
| F1 Score     | nan |
+--------------+---------+


C:\Users\12084\AppData\Local\Temp\ipykernel_17076\3056985081.py:9: RuntimeWarning: invalid value encountered in scalar divide
  return (TP + TN) / (TP + TN + FP + FN)
C:\Users\12084\AppData\Local\Temp\ipykernel_17076\3056985081.py:12: RuntimeWarning: invalid value encountered in scalar divide
  return TN / (TN + FP)
C:\Users\12084\AppData\Local\Temp\ipykernel_17076\3056985081.py:15: RuntimeWarning: invalid value encountered in scalar divide
  return TP / (TP + FN)
C:\Users\12084\AppData\Local\Temp\ipykernel_17076\3056985081.py:18: RuntimeWarning: invalid value encountered in scalar divide
  precision = TP / (TP + FP)
C:\Users\12084\AppData\Local\Temp\ipykernel_17076\3056985081.py:19: RuntimeWarning: invalid value encountered in scalar divide
  recall = TP / (TP + FN)


In [ ]:
def compute_cost(X, y, theta):
    m = len(y)
    cost = (1/2*m) * np.sum(np.square(X.dot(theta) - y))
    return cost

def gradient_descent(X, y, theta, learning_rate, iterations):
    m = len(y)
    cost_history = np.zeros(iterations)
    
    for i in range(iterations):
        theta = theta - (1/m) * learning_rate * (X.T.dot(X.dot(theta) - y))
        cost_history[i] = compute_cost(X, y, theta)
    
    return theta, cost_history

# initial coeffx w/intercept)
theta = np.zeros(X.shape[1])

# abritrary gradient-descent settings
iters = 200
lr = 0.01

# Run gradient-descent
theta, cost_history = gradient_descent(X_train, y_train, theta, lr, iters)

pred = X_test.dot(theta)
mae = np.mean(np.abs(pred - y_test))
mse = np.mean((pred - y_test)**2)
rmse = np.sqrt(np.mean((pred - y_test)**2))
ss_res = np.sum((pred - y_test)**2)
ss_tot = np.sum((y_test - np.mean(y_test))**2)
r2 = 1 - (ss_res / ss_tot)

print("mean squared error: ",mse*100)